<center> 
    <h1> Secondary market for cars in Bulgaria </h1>
    <h3>Ivan Svilenov Penchev</h3>
</center>





<center> <h4> Synopsis </h4></center>
The author collected data for the period of one month from a popular Bulgarian used car market portal. Analysed it using various  methods and tried to determine: 1) what is the current state of the second hand car market in Bulgaria and 2) are cars bought in the second hand car market more fuel cost friendly.

---

### Introduction

Cars and the Bulgarian mentality have always been an interesting mix. While, worldwide, countries are obbying for the use of new cars which are more energy efficient, Bulgaria's favorite choice for cars seems to be the secondary market. 

> Only for the last year the number of bought and registered new cars was 26 000, while the imported used cars from the EU was 266 000. [1]

This of course was the case for 2017, if we compare it to Q1 of 2019 we see an improvement by 29% [2] The number seems substantial, however it is not when we compare the rate of vehicles per capita.


Make a long story short the 9th poorest country in Europe by GDP per capita in international dollars prefers to buy more used cars than new ones.


<div style='float: center'>
  <img style='width: 200px' src="./images/01.png"></img>
</div>

This in itself, while it contributes to the expansion of the biggest in human's  history, environmental crisis, is not something disturbing. Notwithstanding, when the lowest among goods markets in three consecutive years, which it also registered the worst performance of all goods markets on problems experienced by consumers and on comparability[3] is the country's first choice for a market a further analysis and monitoring of said market is needed. 

Based on the above mentioned, we compose the two leading areas of interest of this paper:
1. What is the current state of the second hand car market in Bulgaria?
2. Are cars bought in the second hand car market more fuel cost friendly? 

### Data acquisition 
A cornerstone of science is the possibility to critically assess the correctness of the scientific claims  made and conclusions drawn by other scientists.

In order for this to be possible a systematic approach to and precise description of experimental procedure and subsequent data analysis is needed.

One the first thing we do for enabling reproducibility of results for further analysis and validation by other scientistsis describing how we acquired our data.

For this paper the primary source of information was the website cars.bg, where a webscraping approach was used, this approach is visually described in the picture bellow, while the full code is included in [in this folder](./webscapper_cars.bg).

<div style='float: center'>
  <img src="./images/02.png"></img>
</div>

The data we receive is not 100% clean and further work on it will be needed.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
from enum import Enum
import datetime
import re
from plotnine import *
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from scipy.stats import ttest_ind
import nose.tools

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

In [ ]:
initial_car_df_from_cars_bg = pd.read_json('./data/json_data.txt', 'r', encoding='utf-8-sig')

In [ ]:
nose.tools.assert_is_not_none(initial_car_df_from_cars_bg)
nose.tools.assert_equal(initial_car_df_from_cars_bg.shape, (87900, 6))

In [ ]:
initial_car_df_from_cars_bg = initial_car_df_from_cars_bg.drop("py/object", axis=1)
initial_car_df_from_cars_bg.rename(
    columns={
        "_car_data": "car_data",
        "_car_extra_data": "car_extra_data",
        "_name": "name",
        "_price": "price",
        "_id": "id"
    },
    inplace=True
)


In [ ]:
nose.tools.assert_equal(initial_car_df_from_cars_bg.shape, (87900, 5))

In [ ]:
class Month(Enum):
    Декември = 12
    Ноември = 11
    Октомври = 10
    Септември = 9
    Август = 8
    Юли = 7
    Юни = 6
    Май = 5
    Април = 4
    Март = 3
    Февруари = 2
    Януари = 1

initial_car_df_car_data = pd.io.json.json_normalize(initial_car_df_from_cars_bg['car_data'].values)

def car_data_cleaning_data(chars, varName):
    if type(chars) is str and varName=='Брой врати':
        return chars.split('/')[0]
    if type(chars) is str and varName=='Пробег':
        return int(chars.split(' ')[0].replace(",", ""))
    if type(chars) is str and varName=='Кубатура':
        return int(chars.split(' ')[0].replace(",", ""))
    if type(chars) is str and varName=='Мощност':
        return int(chars.split(' ')[0].replace(",", ""))
    if type(chars) is str and varName=='Година':
        month = Month[chars.split(' ')[0]]
        year = int(chars.split(' ')[1])
        return datetime.datetime(year, month.value, 1)


initial_car_df_car_data['Брой врати'] = initial_car_df_car_data.apply(lambda row: car_data_cleaning_data(row['Брой врати'],'Брой врати'), axis=1)
initial_car_df_car_data['Пробег'] = initial_car_df_car_data.apply(lambda row: car_data_cleaning_data(row['Пробег'],'Пробег'), axis=1)
initial_car_df_car_data['Пробег'] = initial_car_df_car_data['Пробег'].fillna(0)
initial_car_df_car_data['Пробег'] = initial_car_df_car_data['Пробег'].astype(int)

initial_car_df_car_data['Кубатура'] = initial_car_df_car_data.apply(lambda row: car_data_cleaning_data(row['Кубатура'],'Кубатура'), axis=1)
initial_car_df_car_data['Кубатура'] = initial_car_df_car_data['Кубатура'].fillna(0)
initial_car_df_car_data['Кубатура'] = initial_car_df_car_data['Кубатура'].astype(int)

initial_car_df_car_data['Мощност'] = initial_car_df_car_data.apply(lambda row: car_data_cleaning_data(row['Мощност'],'Мощност'), axis=1)
initial_car_df_car_data['Мощност'] = initial_car_df_car_data['Мощност'].fillna(0)
initial_car_df_car_data['Мощност'] = initial_car_df_car_data['Мощност'].astype(int)

initial_car_df_car_data['Година'] = initial_car_df_car_data.apply(lambda row: car_data_cleaning_data(row['Година'],'Година'), axis=1)

initial_car_df_car_data.head(4)

In [ ]:
nose.tools.assert_equal(car_data_cleaning_data('4/5 врати', 'Брой врати'), '4')
nose.tools.assert_equal(car_data_cleaning_data('148,291 км', 'Пробег'), 148291)
nose.tools.assert_equal(car_data_cleaning_data('140 к.с', 'Мощност'), 140)
nose.tools.assert_equal(car_data_cleaning_data('Декември 2010', 'Година'), datetime.datetime(2010, 12, 1, 0, 0))

In [ ]:
initial_car_df_from_cars_bg = pd.concat([initial_car_df_from_cars_bg, initial_car_df_car_data], axis=1, sort=False)
initial_car_df_from_cars_bg = initial_car_df_from_cars_bg.drop(['car_data'], axis=1)
initial_car_df_from_cars_bg.head(3)

In [ ]:
nose.tools.assert_equal(initial_car_df_from_cars_bg.shape[1], 12)

In [ ]:
all_car_brands_df = pd.read_csv('./data/car_brands.csv', header=None, usecols=[0,2])
all_car_brands_df.columns = ['brand_name','region']
all_car_brands_df.head()

def findCarBrand(c):
    if type(c) is str:
        first = c.split(' ')
        try:
            brand_row = all_car_brands_df[all_car_brands_df.brand_name==first[0].replace('-','').replace(' ','')]
            valToReturn = brand_row.brand_name.item()
            return valToReturn
        except ValueError:
            try:
                brand_row = all_car_brands_df[all_car_brands_df.brand_name==first[0].strip()+first[1].strip()]
                valToReturn = brand_row.brand_name.item()
                return valToReturn
            except ValueError:
                return pd.np.NaN
        

initial_car_df_from_cars_bg['car_brand'] = initial_car_df_from_cars_bg.apply(lambda row: findCarBrand(row['name']), axis=1)


In [ ]:
def isSold(r):
    httpCon = HttpConnection()
    url = 'https://www.cars.bg/offer/'+r
    r = httpCon.requestSession.get(url)
    if r.headers.get('Set-Cookie') is None:
        print("id " + url + ' is SOLD')
        return True
    else:
        print("id " + url + ' NOT SOLD')
        return False
    
#Sync call, takes forever to finish, Python paralel HOW?!? 
#initial_car_df_from_cars_bg["sold"] = initial_car_df_from_cars_bg['id'].apply(lambda row: isSold(row))
# with open('car_data_with_brand_and_sold_status.json', 'w', encoding='utf-8') as file:
#      df.to_json(file, force_ascii=False, date_format="iso")
print("Can't put your memes in code cell since they are images?")
image = mpimg.imread("images/03.png")
plt.imshow(image)
plt.title("Python and parallel HTTP requests")
plt.axis('off')
plt.show()
image = mpimg.imread("images/04.jpg")
plt.imshow(image)
plt.axis('off')
plt.title("me trying import asyncio and using event_loop()")
plt.show()



In [ ]:
initial_car_df_from_cars_bg = pd.read_json('data/car_data_with_brand_and_sold_status.json', 'r', encoding='utf-8-sig')
initial_car_df_from_cars_bg['Година'] = pd.to_datetime(initial_car_df_from_cars_bg['Година'])
initial_car_df_from_cars_bg['Брой врати'] = initial_car_df_from_cars_bg['Брой врати'].fillna(0)
initial_car_df_from_cars_bg['Брой врати'] = initial_car_df_from_cars_bg['Брой врати'].astype(int)
df_car_models = pd.read_csv('Data/car_brands_and_models.csv', header=None, usecols=[1,2])
df_car_models.columns = ['brand_name', 'model_name']

def matchCarOfferNameWithCarModel(name, brand):
    name = name
    brandName= brand
    all_models_list = df_car_models[df_car_models['brand_name'].str.contains(brandName)].model_name.to_list()
    bestMatchModel = ''
    for model in all_models_list:
        for match in re.finditer(model, name):
            s = match.start()
            e = match.end()
            lenght = e-s
            if lenght > len(bestMatchModel):
                bestMatchModel=model
    return pd.np.NaN if len(bestMatchModel)==0 else bestMatchModel
initial_car_df_from_cars_bg['car_model'] = initial_car_df_from_cars_bg[initial_car_df_from_cars_bg['car_brand'].isna()==False].apply(lambda row: matchCarOfferNameWithCarModel(row['name'], row['car_brand']), axis=1)



In [ ]:
initial_car_df_car_extra_data = pd.io.json.json_normalize(initial_car_df_from_cars_bg['car_extra_data'].values)
all_posibile_dummy_variable_names = {}
def saveDummyVariableToDictionary(resultList, categoryResultComesFrom, dictionaryToSaveTo):
    for result in resultList:
        if categoryResultComesFrom.lower()+"_"+result.lower().replace(".", " ").replace("  ", " ").replace(" ", "_") not in dictionaryToSaveTo:
             dictionaryToSaveTo[categoryResultComesFrom.lower()+"_"+result.lower().replace(".", " ").replace("  ", " ").replace(" ", "_")]=categoryResultComesFrom.lower()+"_"+result.lower().replace(".", " ").replace("  ", " ").replace(" ", "_")
        
def findAllPossibleDummyVariables(row):
    if type(row['Друго']) is str:
        if len(row['Друго'])>0:
            resultOther = [x.strip() for x in row['Друго'].split(',')]
            saveDummyVariableToDictionary(resultOther, 'Друго',all_posibile_dummy_variable_names)
    if type(row['Комфорт']) is str:        
        if len(row['Комфорт'])>0:
            resultComfort = [x.strip() for x in row['Комфорт'].split(',')]
            saveDummyVariableToDictionary(resultComfort, 'Комфорт',all_posibile_dummy_variable_names)
    if type(row['Сигурност']) is str:        
        if len(row['Сигурност'])>0:
            resultSecurity = [x.strip() for x in row['Сигурност'].split(',')]  
            saveDummyVariableToDictionary(resultSecurity, 'Сигурност',all_posibile_dummy_variable_names)
    
initial_car_df_car_extra_data.apply(lambda row: findAllPossibleDummyVariables(row), axis=1)
print("apply returns a series, which Jupyter displays, this is here to hide it")

In [ ]:

def setKeyForIndexTo(dataframe, index, key, value):
    dataframe.at[index,key] = value
    
def populateDummyVariablesForDataFrame(dataframe, dictionaryDummyVariables):
    for key in dictionaryDummyVariables.keys():
        dataframe[key] = False
    
    for i, row in dataframe.iterrows():
        keyToSetForThisRow = []
        if type(row['Друго']) is str and type(row['Друго']) is not float:
            if len(row['Друго'])>0:
                resultOther = [x.strip() for x in row['Друго'].split(',')]
                for result in resultOther:
                    key = "друго"+"_"+result.lower().replace(".", " ").replace("  ", " ").replace(" ", "_")
                    keyToSetForThisRow.append(key)
        if type(row['Комфорт']) is str and type(row['Комфорт']) is not float:
            if len(row['Комфорт'])>0:
                resultComfort = [x.strip() for x in row['Комфорт'].split(',')]
                for result in resultComfort:
                    key = "комфорт"+"_"+result.lower().replace(".", " ").replace("  ", " ").replace(" ", "_")
                    keyToSetForThisRow.append(key)            
        if type(row['Сигурност']) is str and type(row['Сигурност']) is not float:
            if len(row['Сигурност'])>0:
                resultSecurity = [x.strip() for x in row['Сигурност'].split(',')]
                for result in resultSecurity:
                    key = "сигурност"+"_"+result.lower().replace(".", " ").replace("  ", " ").replace(" ", "_")
                    keyToSetForThisRow.append(key)
        if(len(keyToSetForThisRow)>0):
            for key in keyToSetForThisRow:
                setKeyForIndexTo(dataframe,i, key, True)
            
populateDummyVariablesForDataFrame(initial_car_df_car_extra_data,all_posibile_dummy_variable_names) 
initial_car_df_car_extra_data['Евростандарт'] = initial_car_df_car_extra_data['Евростандарт'].map({'EURO 6': 6, 'EURO 5': 5, 'EURO 4': 4, 'EURO 3': 3, 'EURO 2': 2, 'EURO 1': 1, 0:0})
initial_car_df_car_extra_data = initial_car_df_car_extra_data.drop(['Друго','Комфорт','Описание','Сигурност'], axis=1)


    

In [ ]:
initial_car_df_from_cars_bg = pd.concat([initial_car_df_from_cars_bg, initial_car_df_car_extra_data], axis=1, sort=False)
initial_car_df_from_cars_bg = initial_car_df_from_cars_bg.drop(['car_extra_data'], axis=1)

In [ ]:
#Човекът е сбъркал модела в описанието
initial_car_df_from_cars_bg.at[57257, 'car_model'] = 'K2700'
#Човекът е писал като модел "всички"
initial_car_df_from_cars_bg.at[24860, 'car_model'] = '2'
#Човекът е писал в описанието Mecedes Benz, a е имал предвид субару
initial_car_df_from_cars_bg.at[25085, 'car_model'] = 'Impreza'
initial_car_df_from_cars_bg.at[25085, 'car_brand'] = 'Subaru'
#Не засича, заради +2?
initial_car_df_from_cars_bg.at[122, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[11317, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[23612, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[24193, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[24865, 'car_model'] = 'X-Trail'
initial_car_df_from_cars_bg.at[30938, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[39948, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[40151, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[45074, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[45124, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[52217, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[52649, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[53621, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[55451, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[58082, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[70996, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[72357, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[73743, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[76857, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[76948, 'car_model'] = 'Qashqai+2'
initial_car_df_from_cars_bg.at[78863, 'car_model'] = 'Qashqai+2'
#
initial_car_df_from_cars_bg.at[78564, 'car_model'] = 'Golf'
initial_car_df_from_cars_bg.at[39299, 'car_model'] = '13'
#Range rover
initial_car_df_from_cars_bg.at[31416, 'car_model'] = 'P38'
initial_car_df_from_cars_bg.at[26041, 'car_model'] = 'Sport'
initial_car_df_from_cars_bg.at[24826, 'car_model'] = 'Vogue'
initial_car_df_from_cars_bg.at[56465, 'car_model'] = 'Vogue'
initial_car_df_from_cars_bg.at[85213, 'car_model'] = 'Vogue'
initial_car_df_from_cars_bg.at[37829, 'car_model'] = 'Vogue'
initial_car_df_from_cars_bg.at[38367, 'car_model'] = 'Vogue'

initial_car_df_from_cars_bg = initial_car_df_from_cars_bg[pd.notnull(initial_car_df_from_cars_bg['car_model'])]

DieselAbbr = ['TDI','CDTI','dCi']                
for i, row in initial_car_df_from_cars_bg[(initial_car_df_from_cars_bg['Гориво'].isna()==True) & (initial_car_df_from_cars_bg['name'].str.contains('|'.join(DieselAbbr), flags=re.I)==True)].iterrows():
    setKeyForIndexTo(initial_car_df_from_cars_bg,i, 'Гориво', 'Дизел')
for i, row in initial_car_df_from_cars_bg[(initial_car_df_from_cars_bg['Гориво'].isna()==True)].iterrows():
    setKeyForIndexTo(initial_car_df_from_cars_bg,i, 'Гориво', 'Бензин')   
    #Range Rover e модел... на LandRover
for i, row in initial_car_df_from_cars_bg[(initial_car_df_from_cars_bg['car_brand']=='RangeRover')].iterrows():
    proper_model = 'Range Rover '+row['car_model'].strip();
    setKeyForIndexTo(initial_car_df_from_cars_bg,i, 'car_brand', 'LandRover')
    setKeyForIndexTo(initial_car_df_from_cars_bg,i, 'car_model', proper_model)
    #Volga e модел...
for i, row in initial_car_df_from_cars_bg[(initial_car_df_from_cars_bg['car_brand']=='Volga')].iterrows():
    setKeyForIndexTo(initial_car_df_from_cars_bg,i, 'car_brand', 'GAZ')
for i, row in initial_car_df_from_cars_bg[(initial_car_df_from_cars_bg['car_brand']=='Haval')].iterrows():
    setKeyForIndexTo(initial_car_df_from_cars_bg,i, 'car_brand', 'Greatwall')    


In [ ]:
print('Code below is dependant on searching within strings in a dataframe columns and it is very slow')

# df_car_models_efficiency = pd.read_csv('Data/car_efficiency.txt')
# df_car_models_efficiency['yearStart'] = df_car_models_efficiency['yearStart'].astype('Int32')
# df_car_models_efficiency['yearStop'] = df_car_models_efficiency['yearStop'].astype('Int32')
# ##Checks wheter a dataFrame contains carBrand and carModel in the fields "carBrand" and "carModelGeneration"
# def isModelAndBrandInDataFrame(dataframeToCheck,carBrand, carModel, carYear):
#     carModel=carModel.strip()
#     if((carModel in all_unique_models) == False):
#         all_models_brand = dataframeToCheck[dataframeToCheck['carBrand'].str.contains(carBrand) & dataframeToCheck['carModelGeneration'].str.contains(carModel)]
#         if(all_models_brand.carModel.count()>0):
#             for i, row in all_models_brand.iterrows():
#                  setKeyForIndexTo(dataframeToCheck,i, 'car_model_fk', carModel)
# ##Checks wheter a dataFrame contains carBrand and carModel in the fields "carBrand" and "engine"
# def isModelAndBrandInDataFrameEngine(dataframeToCheck,carBrand, carModel, carYear):
#     carModel=carModel.strip()
#     if((carModel in all_unique_models) == False):
#         all_models_brand = dataframeToCheck[dataframeToCheck['carBrand'].str.contains(carBrand) & dataframeToCheck['engine'].str.contains(carModel)]
#         if(all_models_brand.carModel.count()>0):
#             for i, row in all_models_brand.iterrows():
#                  setKeyForIndexTo(dataframeToCheck,i, 'car_model_fk', carModel)
                    
# df_car_models_efficiency['car_model_fk'] = "unknown"
# initial_car_df_from_cars_bg.apply(lambda row: isModelAndBrandInDataFrame(df_car_models_efficiency, row['car_brand'], row['car_model'], row['Година']), axis=1)
# initial_car_df_from_cars_bg.apply(lambda row: isModelInDataFrameEngine(df_car_models_efficiency, row['car_brand'], row['car_model'], row['Година']), axis=1)
# ##Formats the value in format 0.0,removes symbols that make the conversion to float not possible
# def formatUrban(row):
#     if(type(row['fuelConsumptionUrban']) is str):
#         if(len(re.findall('-', row['fuelConsumptionUrban']))):
#             return row['fuelConsumptionUrban'].split('-')[1]
#         if(len(re.findall('/', row['fuelConsumptionUrban']))):
#             return row['fuelConsumptionUrban'].split('/')[1]
#         if(len(re.findall('.', row['fuelConsumptionUrban']))):
#             return ".".join(row['fuelConsumptionUrban'].split(".", 2))[:2]
#         return row['fuelConsumptionUrban']
# def formatUrbanComb(row):
#     if(type(row['fuelConsumptionCombined']) is str):
#         if(len(re.findall('-', row['fuelConsumptionCombined']))):
#             return row['fuelConsumptionCombined'].split('-')[1]
#         if(len(re.findall('/', row['fuelConsumptionCombined']))):
#             return row['fuelConsumptionCombined'].split('/')[1]
#         if(len(re.findall('.', row['fuelConsumptionCombined']))):
#             return ".".join(row['fuelConsumptionCombined'].split(".", 2))[:2]
#         return row['fuelConsumptionCombined']
# df_car_models_efficiency['fuelConsumptionUrban']=df_car_models_efficiency.apply(lambda row: formatUrban(row), axis=1)
# df_car_models_efficiency['fuelConsumptionCombined']=df_car_models_efficiency.apply(lambda row: formatUrbanComb(row), axis=1)
# df_car_models_efficiency['fuelConsumptionCombined'] = df_car_models_efficiency['fuelConsumptionCombined'].astype('float64')
# df_car_models_efficiency['fuelConsumptionUrban'] = df_car_models_efficiency['fuelConsumptionUrban'].astype('float64')

# grBrand = df_car_models_efficiency.groupby('carBrand')
# grModel = df_car_models_efficiency.groupby('car_model_fk')
# modelUbran = grModel.fuelConsumptionUrban.apply(lambda x: x.median())
# modelCombined = grModel.fuelConsumptionCombined.apply(lambda x: x.median())
# brandUbran = grBrand.fuelConsumptionUrban.apply(lambda x: x.median())
# brandCombined = grBrand.fuelConsumptionCombined.apply(lambda x: x.median())
# ##Creates a multi dictionary for easy searching 
# cacheModel = dict()
# cacheBrand = dict()
# def createFuelConsumptionDictionaryFromSeries(dictionaryToSaveTo, seriesToIterate, varibleNameToSave):
#     for items in seriesToIterate.iteritems():
#         key = items[0].strip()
#         if (key is 'unknown'):
#             continue
#         if key in dictionaryToSaveTo:
#             dictionaryToSaveTo[key].update({varibleNameToSave : items[1]})
#         else:
#             dictionaryToSaveTo[key] = {varibleNameToSave:items[1]}         

# createFuelConsumptionByModelDictionary(cacheModel,modelUbran, 'fuelConsumptionUrban')
# createFuelConsumptionByModelDictionary(cacheModel,modelCombined, 'fuelConsumptionCombined')
# createFuelConsumptionByModelDictionary(cacheBrand,brandUbran, 'fuelConsumptionUrban')
# createFuelConsumptionByModelDictionary(cacheBrand,brandCombined, 'fuelConsumptionCombined')
# ##Searches for value by model first if not found by brand, if still not found assigns default value 
# def assignFuelConsuption(carModel, carBrand):
#     carModel = carModel.strip()
#     carBrand = carBrand.strip()
#     try:
#             value = cacheModel[carModel]
#             fuelConsumptionUrban=value['fuelConsumptionUrban']
#             fuelConsumptionCombined=value['fuelConsumptionCombined']
#             if((pd.isna(fuelConsumptionUrban)==False) and (pd.isna(fuelConsumptionCombined)==False)):
#                 return max(fuelConsumptionUrban, fuelConsumptionCombined)
#             if((pd.isna(fuelConsumptionUrban)==False) and (pd.isna(fuelConsumptionCombined))):
#                 return fuelConsumptionUrban
#             if((pd.isna(fuelConsumptionUrban)) and (pd.isna(fuelConsumptionCombined)==False)):
#                 return fuelConsumptionCombined
#             if((pd.isna(fuelConsumptionUrban)) and (pd.isna(fuelConsumptionCombined))):
#                 return fuelConsumptionUrban
#     except:
#         try:
#             value = cacheBrand[carBrand]
#             fuelConsumptionUrban=value['fuelConsumptionUrban']
#             fuelConsumptionCombined=value['fuelConsumptionCombined']
#             if((pd.isna(fuelConsumptionUrban)==False) and (pd.isna(fuelConsumptionCombined)==False)):
#                 return max(fuelConsumptionUrban, fuelConsumptionCombined)
#             if((pd.isna(fuelConsumptionUrban)==False) and (pd.isna(fuelConsumptionCombined))):
#                 return fuelConsumptionUrban
#             if((pd.isna(fuelConsumptionUrban)) and (pd.isna(fuelConsumptionCombined)==False)):
#                 return fuelConsumptionCombined
#             if((pd.isna(fuelConsumptionUrban)) and (pd.isna(fuelConsumptionCombined))):
#                 return fuelConsumptionUrban
#         except:
#             return 100.00
# initial_car_df_from_cars_bg['fuel_consumption'] = initial_car_df_from_cars_bg.apply(lambda row: assignFuelConsuption(row['car_model'], row['car_brand']), axis=1)
# final_car_df = initial_car_df_from_cars_bg
# ##Add carBrand Region
# all_car_brands_df.set_index('brand_name', inplace = True)
# car_brand_dic = all_car_brands_df.to_dict('index')
# def findCarBrandOriginRegion(carBrand):
#     if(carBrand in car_brand_dic):
#         return car_brand_dic[carBrand]['region']
#     else:
#         return pd.np.NaN
# final_car_df['car_brand_region'] = final_car_df.apply(lambda row: findCarBrandOriginRegion(row['car_brand']), axis=1)
# ##Set default value for cars that run on electricity and we have not a fuel_conmption for them, mostly Teslas
# ##Number is based on [6]
# for i, value in final_car_df[(final_car_df['Гориво']=='Електричество')&((pd.isna(final_car_df.fuel_consumption)==True) |(final_car_df.fuel_consumption==100))].iterrows():
#     setKeyForIndexTo(final_car_df,i,'fuel_consumption',14.4)
# ##Exclude cars with default value or cars without value or with default value of 100 
# final_car_df_no_def = final_car_df[((final_car_df.fuel_consumption==100)==False)]
# final_car_df_no_def = final_car_df_no_def[(pd.isna(final_car_df_no_def.fuel_consumption)==False)]
# ##Save intermediate file  
# with open('final_final_final_car_data.json', 'w', encoding='utf-8') as file:
#       final_car_df_no_def.to_json(file, force_ascii=False, date_format="iso")
     

### Data acquisition - Summary 
In the current section we created a [webscraper](./webscapper_cars.bg) which we used to generate 3 datasets:
1. [json_data.txt](./Data);
2. [car_brands.csv](./Data);
3. and [car_brands_and_models.csv](./Data).

Those datasets were further cleaned and combined. [json_data.txt](./Data) contained two nested dictionaries. Initially we started to work with the first one, once the data were extracted and plotted it in the data frame, where  variables like 'Брой врати', 'Пробег', 'Кубатура', 'Мощност', and 'Година' were properly formatted a further converted into the appropriate format. For example 'Брой врати' was converted from string of type '4/5' to an int. Later on we used the name of the car offer to determain the car brand.

Once that was done, due to the long duration of the operation, it was time to check, the status of the car offers and how many of them were sold.

Finishing with that process led us to look in the second nested dictionary, where we created a multitude of dummy variables based on sub embedded csv string.

To finish things with [json_data.txt](./Data), further examination of the data was done and some manual corrections were inserted, where incorrect data was spotted. 

After some inquiry a dataset [car_efficency.xml](./Data) with manufacturer presented information was acquired[5] For the needs of using the dataset in pandas an [XmlToCsv](./XmlToJsonParser) parser was created, which led to the creation of the last used dataset:
4. [car_efficency.csv](./Data)

Once the dataset was imported a link between the two datasets (car_model_fk) was created for easier searching. After some formatting of fuelConsumptionUrban and fuelConsumptionCombined variables in proper float values, it was calculated fuelConsumption for each known car model and car brand. That data was transpired in the original data frame, where if a match cannot be created, a default value was inserted, later that the default value was dropped.

The end result can be summarised by comparing the initial shape of the data and the consequent one. We started with 87900 observations, which were later reduced to 87894, while the starting variable were increased from 6 to 61.

> Remark: The author has created and used some intermediate files, namely [car_data_with_brand_and_sold_status.json](./Data) and [final_final_car_data.json](./Data), in order to increase speed during reproducibility.

### Data analysis 
In this section we will attempt to dissect the data and answer the initial questions set by this inquiry for knowledge.

In [ ]:
car_offers_df = pd.read_json('data/final_final_final_car_data.json', 'r', encoding='utf-8-sig')
car_offers_df['Година'] = pd.to_datetime(car_offers_df['Година'])

In [ ]:
print('We have data for {} car offer ads, which contains {} unique car brands and {} unique car models'.format(car_offers_df.shape[0], car_offers_df.car_brand.nunique(), car_offers_df.car_model.nunique()))
print('The oldest car model we have is from {} year, while the newest is from {}'.format(car_offers_df['Година'].min().year, car_offers_df['Година'].max().year))

This seems a bit weird, we are interested in second hand cars, yet we see models from 2019? We shall exclude them from the data.

In [ ]:
print('We {} car that have millege of 0, and {} cars that are from 2019, the are overlapping, are from 2019 with and with 0 mileage, {} times.'.format(car_offers_df[car_offers_df['Пробег']==0].id.count(),car_offers_df[car_offers_df['Година'].dt.year==2019].id.count(),car_offers_df[(car_offers_df['Година'].dt.year==2019) & (car_offers_df['Пробег']==0)].id.count()))


In [ ]:
car_offers_df = car_offers_df[~((car_offers_df['Година'].dt.year==2019))]
car_offers_df = car_offers_df[~((car_offers_df['Пробег']==0))]
## sanity checks, not impossible, however highly unlikely
car_offers_df = car_offers_df[~(car_offers_df['Пробег']>1000000)]
car_offers_df = car_offers_df[~(car_offers_df['Мощност']>700)]
car_offers_df= car_offers_df[~(car_offers_df['Година'].isnull())]
#forgotten temp variable
car_offers_df = car_offers_df.drop(['Found'], axis=1)
#Дизел - 2.33 BGN
#Бензин - 2.22 BGN
#Газ/Бензин - 0.97+Benzin/2  BGN
#Метан/Бензин - 1.47+Benzin/2 BGN
#Хибрид - Dizel + El / 2 BGN
#Електричество - 0.14 BGN
#As per [7]'s fig 4.3 on avarage 40 km is the daily travel distance by car. The average month has 365/12 = 30.42 days in.
pricedFuelTypes = {
		"Дизел" : 2.33,
		"Бензин" : 2.22,
		"Газ/Бензин" : 1.60,
		"Метан/Бензин" : 1.80,
		"Хибрид" : 1.24,
		"Електричество" : 0.14
	}
def computeMonthlyFuelExpenditurePerCar(fuelType, fuelExpenditure):
    if(fuelType in pricedFuelTypes):
        #fuelExpenditure is per 100KM
        return ((40*30.42)/100)*fuelExpenditure;
    else:
        return pd.np.NaN
    
car_offers_df['montly_fuel_expenditure']=car_offers_df.apply(lambda row: computeMonthlyFuelExpenditurePerCar(row['Гориво'],row['fuel_consumption']), axis=1)

In [ ]:
car_offers_df.describe().T

In [ ]:
cols = list(['price','Кубатура', 'Мощност', 'Пробег'])

def calculateZScoreForColInDataframe(listCol, dataframe):
    for col in listCol:
        col_zscore = col + '_zscore'
        dataframe[col_zscore] = (dataframe[col] - dataframe[col].mean())/dataframe[col].std(ddof=0)


calculateZScoreForColInDataframe(cols,car_offers_df)

In [ ]:
print('We have abnormal Z-Score for {} offers [mileage], {} offers [cubature], {} offers [horse power] and {}  offers [price]'.format(
    car_offers_df[np.abs(car_offers_df['Пробег_zscore'])>3].id.count(),
    car_offers_df[np.abs(car_offers_df['Кубатура_zscore'])>3].id.count(),
    car_offers_df[np.abs(car_offers_df['Мощност_zscore'])>3].id.count(),
    car_offers_df[np.abs(car_offers_df['price_zscore'])>3].id.count()))

car_offers_df = car_offers_df[np.abs(car_offers_df['Пробег_zscore'])<3]
car_offers_df = car_offers_df[np.abs(car_offers_df['Кубатура_zscore'])<3]
car_offers_df = car_offers_df[np.abs(car_offers_df['Мощност_zscore'])<3]

car_offers_df = car_offers_df.drop(['Пробег_zscore'], axis=1)
car_offers_df = car_offers_df.drop(['Кубатура_zscore'], axis=1)
car_offers_df = car_offers_df.drop(['Мощност_zscore'], axis=1)
car_offers_df = car_offers_df.drop(['price_zscore'], axis=1)


In [ ]:
car_offers_df.describe().T

In [ ]:
car_offers_df.describe(include="bool").T

In [ ]:
plt.figure(figsize = (6, 4))
min_month, max_month = int(car_offers_df.montly_fuel_expenditure.min()), int(car_offers_df.montly_fuel_expenditure.max())

car_offers_df.montly_fuel_expenditure.hist(bins=20)
plt.title('Histogram of Montly expenditure')
plt.xlabel('Expenditure per month in BGN')
plt.ylabel('Frequency')
plt.xticks(range(min_month, max_month + 10, 10), rotation='vertical')

plt.show()

In [ ]:
plt.figure(figsize = (6, 4))
car_offers_per_region=car_offers_df.groupby(car_offers_df.car_brand_region).size()
plt.bar(car_offers_per_region.index, car_offers_per_region, align='center')

plt.title("Number of car offers per car brand region")
plt.xlabel("Car brand origin region")
plt.ylabel("Car Offers")

plt.show()

In [ ]:
offer_counts_by_year = car_offers_df.groupby(car_offers_df['Година'].dt.year).size()
min_year, max_year = int(car_offers_df['Година'].min().year), car_offers_df['Година'].max().year
print('The most common manufacturing year of listed car offers is {} with {} offers, while the least common manufacturing year is {} with {} offers'.format(int(offer_counts_by_year.idxmax()),int(offer_counts_by_year.max()), int(offer_counts_by_year.idxmin()),int(offer_counts_by_year.min())))

plt.figure(figsize = (16, 8))
plt.bar(offer_counts_by_year.index, offer_counts_by_year, align='center')
plt.title("Number of car offers per car model's manufacturing year")
plt.xlabel("Year of manufacturing")
plt.ylabel("Car Offers")

plt.xticks(range(min_year, max_year + 1), rotation='vertical')
plt.show()

In [ ]:
car_brand_list = car_offers_df['car_brand'].value_counts().index.tolist()
car_brand_cat = pd.Categorical(car_offers_df['car_brand'], categories=car_brand_list)

(ggplot(car_offers_df)
 + aes(x='car_brand_cat')
 + geom_bar(size=20, show_legend=True)
 + geom_text(aes(label='stat(count)'),stat='count')
 + geom_text(
     aes(label='stat(prop)*100', group=1),
     stat='count',
     size=8,
     position='fill',
     format_string='({:.1f}%)'
 )
 + scale_x_discrete(limits=car_brand_list, reverse=True)
 + coord_flip()
 + labs(y='Count', x='Brand', title='Number of Car offers by Brand')
 + theme(figure_size=(8, 11))
)


In [ ]:
plt.figure(figsize = (8, 6))
top_cities = car_offers_df.groupby("град").size().sort_values(ascending = False).head(15)
plt.barh(range(len(top_cities.index)), top_cities)

plt.title("Top 15 Cities with car offers")
plt.xlabel("Number of Car Offer listings")
plt.yticks(range(len(top_cities)), top_cities.index)
plt.gca().invert_yaxis()
plt.show()



In [ ]:
(ggplot(car_offers_df)
 + aes(x='sold')
 + geom_bar(size=20, show_legend=True)
 + geom_text(aes(label='stat(count)'),stat='count')
 + geom_text(
     aes(label='stat(prop)*100', group=1),
     stat='count',
     position='fill',
     format_string='({:.1f}%)'
 )

 + labs(y='Count', x='Is car offer sold?', title='Number of Car offer listing that have been marked as Sold')
 + theme(figure_size=(6, 4))
)

In [ ]:
sns.set(rc={'figure.figsize':(15.7,18.27)})
pd.crosstab(car_offers_df.car_brand,car_offers_df.sold).plot(kind='bar')
plt.title('Purchase Frequency for Car Brand')
plt.xlabel('Car Brand')
plt.ylabel('Frequency of Purchase')
plt.ylim(0, 8000)
plt.show()

In [ ]:
(ggplot(data=car_offers_df,
           mapping=aes(x='factor(sold)'
                          ))
    + geom_bar()
    + geom_text(aes(label='stat(count)'),stat='count')
    + facet_wrap("car_brand_region")
    + theme(figure_size=(6, 4))
    + labs(y='Car offer count', x='Is car offer sold?', title='Number of Car offers and their "sold" status by Car Bran Origin region')
)

In [ ]:
(ggplot(data=car_offers_df,
           mapping=aes(x='factor(sold)'
                          ))
    + geom_bar()
    + geom_text(aes(label='stat(count)'),stat='count')
    + facet_wrap("Гориво")
    + theme(figure_size=(6, 4))
    + labs(y='Car offer count', x='Is car offer sold?', title='Number of Car offers and their "sold" status by Fuel type used')
)

In [ ]:
(ggplot(data=car_offers_df[car_offers_df.price<50000],
           mapping=aes(x='price'))
    + geom_histogram()
    + facet_wrap("car_brand_region")
    + theme(figure_size=(10, 4))
    + labs(y='Frequency', x='Price', title='Distribution of the price per car brand region with max price set to 50 000 BGN')
)

In [ ]:
sns.set(rc={'figure.figsize':(15.7,18.27)})

g = sns.boxplot(x="price", y="car_brand",
            data=car_offers_df, orient='h')
#sns.despine(offset=10, trim=True)
plt.xlim(0, 250000)
plt.xticks(np.arange(0, 250000, 5000),rotation=90)
plt.show()

In [ ]:
(ggplot(data=car_offers_df,
           mapping=aes(x='factor(Евростандарт)'
                          ))
    + geom_bar()
    + geom_text(aes(label='stat(count)'),stat='count')
    + theme(figure_size=(6, 4))
    + labs(y='Car offer count', x='Number of Eurostandart the car has', title='Number of Car offers by their Eurostandard Number')
)


In [ ]:
(ggplot(data=car_offers_df,
           mapping=aes(x='factor(частно_лице)'
                          ))
    + geom_bar()
    + geom_text(aes(label='stat(count)'),stat='count')
    + theme(figure_size=(6, 4))
    + labs(y='Car offer count', x='Is it sold by private customer?', title='Number of Car offers by their seller type')
)

In [ ]:
sold_1000_car_listing = car_offers_df[car_offers_df.sold==True].head(1000)
not_sold_1000_car_listing = car_offers_df[car_offers_df.sold==False].head(1000)
ttest_ind(sold_1000_car_listing["montly_fuel_expenditure"], not_sold_1000_car_listing["montly_fuel_expenditure"])

In [ ]:
car_offers_modeling = car_offers_df.filter(['price','Година','Брой врати','Кубатура','Мощност','Пробег','Евростандарт','fuel_consumption', 'sold', 'Гориво', 'montly_fuel_expenditure'], axis=1)
car_offers_modeling['car_man_year']=car_offers_modeling['Година'].dt.year
car_offers_modeling = car_offers_modeling.drop(['Година'], axis=1)

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr = car_offers_modeling.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax,  annot=True,)

In [ ]:
car_offers_modeling.head()

In [ ]:
fuel_type_dummy = pd.get_dummies(car_offers_modeling['Гориво'], prefix='fuel_type')
car_offers_modeling=car_offers_modeling.join(fuel_type_dummy)
car_offers_modeling = car_offers_modeling.drop(['Гориво'], axis=1)
car_offers_modeling.columns.values

In [ ]:
X = car_offers_modeling.loc[:, car_offers_modeling.columns != 'sold']
y = car_offers_modeling.loc[:, car_offers_modeling.columns == 'sold']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

I was thinking about using SMOTE, to balance out the "sold and not sold". However gave up on the idea due to the fact that when RFE was used to select features, it gave me things like "Number of doors" is a very good feature for predicting if a car would be sold.

In [ ]:
# X = car_offers_modeling.loc[:, car_offers_modeling.columns != 'sold']
# y = car_offers_modeling.loc[:, car_offers_modeling.columns == 'sold']


# 
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
# columns = X_train.columns
# os = SMOTE(random_state=0)
# os_data_X,os_data_y=os.fit_sample(X_train, y_train)
# os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
# os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])


# print("length of oversampled data is ",len(os_data_X))
# print("Number of cars offers in not sold state in oversampled data",len(os_data_y[os_data_y['y']==0]))
# print("Number of cars offers in sold state",len(os_data_y[os_data_y['y']==1]))
# print("Proportion of not sold state data in oversampled data is ",len(os_data_y[os_data_y['y']==0])/len(os_data_X))
# print("Proportion of sold state data in oversampled data is ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))

# logreg = LogisticRegression()
# rfe = RFE(logreg, 20)
# rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
# print(rfe.support_)
# print(rfe.ranking_)

In [ ]:
logis = LogisticRegression(solver='lbfgs')
logis.fit(X_train, y_train.values.ravel())
logis_score_train = logis.score(X_train, y_train)
print("Training score: ",logis_score_train)
logis_score_test = logis.score(X_test, y_test)
print("Testing score: ",logis_score_test)
y_pred = logis.predict(X_test)

In [ ]:
coeff_df = pd.DataFrame(X.columns.delete(0))
coeff_df.columns = ['Features']
coeff_df["Coef"] = pd.Series(logis.coef_[0])
coeff_df.sort_values(by='Coef', ascending=False)

In [ ]:
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
print(classification_report(y_test, y_pred))

In [ ]:
logit_roc_auc = roc_auc_score(y_test, logis.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logis.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()


### Data analysis - Summary 
In the current section we loaded the data and upon the initial analysis figured out and some additional filtering and tunning of the data would be needed. Therefore, we removed all new car listings or other listings that have insane values. For example, we had 100 car listings that insisted that the car they offered had more than 700 horse power where a typical Nascar car would have thought. Lastly, we computed what is the monthly fuel cost for each car that was listed.

In the pursuit for our dataset for modeling, after listing some descriptive statistics, we found some abnormalities. We computed a z-score for each of those and removed the one above the recommended values.

Afterwards we did some visual data exploration, tested a hypothesis, failed to use SMOTE, and finally we build and evaluated a logistic regression as our model.

### Key findings
When analysing the Bulgarian secondary market for cars some interisting findings are:
* The price of 75% of the total 85426 cars listed was under 8500 BGN, where the cheapest car was 250 BGN.
* Most cars listed (32319) don't have a Eurostandard, while Eurostandard 4 is the most common Eurostandard with (26786) listings 
* Most sellers on the market were private, however there is a huge amount of entities which are not, the distribution 54% private / 46% not .
* Car brands are present from all continents where dominent are brands from Europe first, second Asia and lastly North America.
* The most common manufacturing year of listed car offers is 2005 with 8217 offers.
* Most car offers are with manufacturing years between 1998 and 2010
* German brands are dominent on the market where VW has 12.5%, Mecedes has 9.3%, BMW has 9.0%, Audi has 8.5% and Opel has 8.3%
* The most dominent Asian brand is Toyota with 5.2% while the most dominent American brand is Chevroulet with 0.8%
* Number of car offer listing follow the city size. That would say the bigger the city the more offers it has. This has one anomality, where "Дупница" not that big of a city is placed within top 5.
* For the period of 1 month, which is the time the author collected data 11-06-2019 to 09-07-2019, out of the 85426  listed cars 12.2% or 10419 were marked sold.
* Where the most sold brands were VW, Mercedes, BMW, Audi, Opel and Peugeout, which consequivently are also the 5 most domminent car brands.
* Most cars that were sold were Europeans (7725).
* Out of all sold cars, 60% run on Dizel, 25% on Benzin, 13% on Gas, 1% on Metan and only 0.001% are electric or Hybrid.
* Prices for the offered cars varried with some brands like Mercedes, Audi, Land Rover having a lot of outliers, however in genereal most prices were under 10 000 BGN
* Cars from Asia and Europe have more members of the class above 10000 BGN visibly from the price distribution for car brands
* The Price correlated strongly with the car's horse power.
* The sold status correlated positively only with the milage of the car and the  fuel consumption, even though the latter had a very close to 0 coefficient, while it negatively correlated with the manufacturing year.
* Eurostandard didn't have much affect.
* The montly fuel expenditure for most car listings were between 84 BGN and 134 BGN
* Most offered cars 63% used Disel as fuel (54596).
* Avarage millage for offered cars was 181098 KM, with diviation of 66285 KM.
* Only 25% of all listings were for cars under 150000 KM.
* Most offered features were Heating windows (90%), Central locking system (88%), Anti-lock braking system (85%), Airbags (84%),  Build-in audio system (81%), Servo Control (78%), Bord computer (68%), Electronic stability control (64%) and Climatronic (58%). 

## Conclussion

The author has set two leading areas of interest of this paper:
1. What is the current state of the second hand car market in Bulgaria?
2. Are cars bought in the second hand car market more fuel cost friendly? 

In regards of the first we can see that Bulgaria has a diverse secondary market for cars which is, judging by the amounth of enterprises, quite lucrative. It has a huge selection of brands, hower it is saturated with older models.The market does not value environmental cars with only 0.002% of all listings being for electric or hybrid cars as well as 37% of all listings not having Euro standard.  While it values mostly Diesel cars with above normal mileage of use and older year of manufacturing. Modern features such as Apple CarPlay and Android Auto,  Adaptive Cruise Control, Automatic Emergency Braking, 360-Degree Camera or Parktronic also fall behind things like Build-in audio system.

In regards to the second statement, there is no significant difference between the expenditure for fuel for cars that are bought and the car that are not bought.

## Study Limitations
The selling of the cars could not be verified, via for example some sort of public API, instead the author considered a car sold if it dissapeared as a listing.

### References
1. [В България се карат най-старите коли в ЕС](https://www.investor.bg/analizi/447/a/v-bylgariia-se-karat-nai-starite-koli-v-es-248385/)
2. [Статистика: 10 пъти по-бедни сме от германците по покупки на нови коли](https://www.dnes.bg/obshtestvo/2019/05/18/statistika-10-pyti-po-bedni-sme-ot-germancite-po-novi-koli.410667)
3. [Study on the second hand cars market](https://ec.europa.eu/info/publications/study-second-hand-cars-market_en)
4. [European second-hand car market analysis](https://www.oeko.de/oekodoc/1114/2011-005-en.pdf)
5. [auto-data.net](https://www.auto-data.net/en/)
6. [Crunching The Numbers: How Much Will Your Electric Car Really Cost?](https://cleantechnica.com/2019/01/11/crunching-the-numbers-how-much-does-your-electric-car-really-cost/)
7. [Driving and parking patterns of
European car drivers --- a mobility survey](https://setis.ec.europa.eu/system/files/Driving_and_parking_patterns_of_European_car_drivers-a_mobility_survey.pdf)